In [25]:
import os
from sklearn.model_selection import train_test_split
import pandas as pd
import mlflow
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from autofeat import AutoFeatClassifier
from sklearn.preprocessing import (
    OneHotEncoder, 
    SplineTransformer, 
    QuantileTransformer, 
    RobustScaler,
    PolynomialFeatures,
    KBinsDiscretizer,
)
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import (
    roc_auc_score, f1_score, precision_score, recall_score, 
    log_loss, confusion_matrix, mean_squared_error, r2_score
)
from catboost import CatBoostClassifier
from dotenv import load_dotenv, find_dotenv
import psycopg
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split, GridSearchCV, RandomizedSearchCV


In [2]:
# подгружаем .env
load_dotenv()

True

In [3]:
TABLE_NAME = "alt_users_churn" # таблица с данными в postgres 
os.environ["MLFLOW_S3_ENDPOINT_URL"] = "https://storage.yandexcloud.net"

# Параметры для трекинга эксперимента
TRACKING_SERVER_HOST = "127.0.0.1"
TRACKING_SERVER_PORT = 5000

EXPERIMENT_NAME = "churn_experiment_imartnv"
RUN_NAME = 'model_grid_search' # ваш код здесь
REGISTRY_MODEL_NAME = "churn_model_martynov_alexey"


# Настрофка отображения
pd.options.display.max_columns = 100
pd.options.display.max_rows = 64

sns.set_style("white")
sns.set_theme(style="whitegrid") 

In [4]:
# Подключение к базе и получение данных
connection = {"sslmode": "require", "target_session_attrs": "read-write"}
postgres_credentials = {
    "host": os.getenv("DB_DESTINATION_HOST"),
    "port": os.getenv("DB_DESTINATION_PORT"),
    "dbname": os.getenv("DB_DESTINATION_NAME"),
    "user": os.getenv("DB_DESTINATION_USER"),
    "password": os.getenv("DB_DESTINATION_PASSWORD"),
}

connection.update(postgres_credentials)

with psycopg.connect(**connection) as conn:

    with conn.cursor() as cur:
        cur.execute(f"SELECT * FROM {TABLE_NAME}")
        data = cur.fetchall()
        columns = [col[0] for col in cur.description]

df = pd.DataFrame(data, columns=columns)

In [5]:
#Функция удаления дубликатов
def remove_duplicates(data):
    feature_cols = data.columns.drop('customer_id').tolist()
    is_duplicated_features = data.duplicated(subset=feature_cols, keep=False)
    data = data[~is_duplicated_features].reset_index(drop=True)
    return data

In [6]:
#Функция для заполнения пропусков
def fill_missing_values(data):
    cols_with_nans = data.isnull().sum()
    cols_with_nans = cols_with_nans[cols_with_nans > 0].index.drop('end_date')
    for col in cols_with_nans:
        if data[col].dtype in [float, int]:
            fill_value = data[col].mean()
        elif data[col].dtype == 'object':
            fill_value = data[col].mode().iloc[0]
        data[col] = data[col].fillna(fill_value)
    return data

In [7]:
#Функция удаления выбросов
def remove_outliers(df: pd.DataFrame, threshold: float = 1.5) -> pd.DataFrame:
        num_cols = df.select_dtypes(include=['float']).columns
        potential_outliers = pd.DataFrame(False, index=df.index, columns=num_cols)
        
        for col in num_cols:
            Q1 = df[col].quantile(0.25)
            Q3 = df[col].quantile(0.75)
            IQR = Q3 - Q1
            margin = threshold * IQR
            lower = Q1 - margin
            upper = Q3 + margin
            potential_outliers[col] = ~df[col].between(lower, upper)
        
        outliers = potential_outliers.any(axis=1)
        df_cleaned = df[~outliers]
        return df_cleaned

In [8]:
#Почистим датасет
df = fill_missing_values(df)
df = remove_duplicates(df)
df = remove_outliers(df)

In [9]:
df = df.set_index('id')
df = df.drop(columns=['customer_id','begin_date','end_date'])

In [10]:
obj_df = df.select_dtypes(include="object")
cat_columns = ["type", "payment_method", "internet_service", "gender"]

In [11]:
num_df = df.select_dtypes(include='float64').dropna()
num_columns = ["monthly_charges", "total_charges"]

n_knots = 3
degree_spline = 4
n_quantiles=100
degree = 3
n_bins = 5
encode = 'ordinal'
strategy = 'uniform'
subsample = None

In [12]:
df.dropna(subset=['monthly_charges','total_charges'],inplace=True)
num_df = df[num_columns]

In [13]:
num_columns = ["monthly_charges", "total_charges"]

n_knots = 3
degree_spline = 4
n_quantiles=100
degree = 3
n_bins = 5
encode = 'ordinal'
strategy = 'uniform'
subsample = None


# SplineTransformer
encoder_spl = SplineTransformer(n_knots=n_knots, degree=degree_spline)
encoded_features = encoder_spl.fit_transform(df[num_columns].to_numpy())

encoded_df = pd.DataFrame(encoded_features, columns=encoder_spl.get_feature_names_out(num_columns))
num_df = pd.concat([num_df, encoded_df], axis=1)


# QuantileTransformer
encoder_q = QuantileTransformer(n_quantiles=n_quantiles)
encoded_features = encoder_q.fit_transform(df[num_columns].to_numpy())

encoded_df = pd.DataFrame(encoded_features, columns=encoder_q.get_feature_names_out(num_columns))

encoded_df.columns = [col + f"_q_{n_quantiles}" for col in num_columns]
num_df = pd.concat([num_df, encoded_df], axis=1)


# RobustScaler
encoder_rb = RobustScaler()
encoded_features = encoder_rb.fit_transform(df[num_columns].to_numpy())

encoded_df = pd.DataFrame(encoded_features, columns=encoder_rb.get_feature_names_out(num_columns))
encoded_df.columns = [col + f"_robust" for col in num_columns]
num_df = pd.concat([num_df, encoded_df], axis=1)


# PolynomialFeatures
encoder_pol = PolynomialFeatures(degree=degree)
encoded_features = encoder_pol.fit_transform(df[num_columns].to_numpy())

encoded_df = pd.DataFrame(encoded_features, columns=encoder_pol.get_feature_names_out(num_columns))
encoded_df.drop(encoded_df.columns[:1 + len(num_columns)], axis=1, inplace=True)
num_df = pd.concat([num_df, encoded_df], axis=1)

# KBinsDiscretizer
encoder_kbd = KBinsDiscretizer(n_bins=n_bins, encode=encode, strategy=strategy, subsample=subsample)
encoded_features = encoder_kbd.fit_transform(df[num_columns].to_numpy())

encoded_df = pd.DataFrame(encoded_features, columns=encoder_kbd.get_feature_names_out(num_columns))

encoded_df.columns = [col + f"_bin" for col in num_columns]
num_df = pd.concat([num_df, encoded_df], axis=1)


num_df.head(2)

,monthly_charges,total_charges,monthly_charges_sp_0,monthly_charges_sp_1,monthly_charges_sp_2,monthly_charges_sp_3,monthly_charges_sp_4,monthly_charges_sp_5,total_charges_sp_0,total_charges_sp_1,total_charges_sp_2,total_charges_sp_3,total_charges_sp_4,total_charges_sp_5,monthly_charges_q_100,total_charges_q_100,monthly_charges_robust,total_charges_robust,monthly_charges^2,monthly_charges total_charges,total_charges^2,monthly_charges^3,monthly_charges^2 total_charges,monthly_charges total_charges^2,total_charges^3,monthly_charges_bin,total_charges_bin
17,20.65,1022.95,0.000000,0.001993,0.183362,0.598305,2.129940e-01,0.003346,0.000000,0.012710,0.321173,0.562876,1.030608e-01,0.000181,0.820707,0.866039,0.458142,1.200842,9072.5625,520507.9125,2.986240e+07,864161.578125,4.957838e+07,2.844394e+09,1.631876e+11,3.0,3.0
59,24.95,894.30,0.037366,0.444730,0.471576,0.046328,2.170599e-08,0.000000,0.031877,0.425038,0.489533,0.053551,7.327987e-07,0.000000,0.040404,0.212313,-0.933763,-0.325363,384.1600,5868.2400,8.964036e+04,7529.536000,1.150175e+05,1.756951e+06,2.683832e+07,0.0,0.0


In [14]:
num_df

,monthly_charges,total_charges,monthly_charges_sp_0,monthly_charges_sp_1,monthly_charges_sp_2,monthly_charges_sp_3,monthly_charges_sp_4,monthly_charges_sp_5,total_charges_sp_0,total_charges_sp_1,total_charges_sp_2,total_charges_sp_3,total_charges_sp_4,total_charges_sp_5,monthly_charges_q_100,total_charges_q_100,monthly_charges_robust,total_charges_robust,monthly_charges^2,monthly_charges total_charges,total_charges^2,monthly_charges^3,monthly_charges^2 total_charges,monthly_charges total_charges^2,total_charges^3,monthly_charges_bin,total_charges_bin
17,20.65,1022.95,0.000000,1.993040e-03,0.183362,0.598305,2.129940e-01,0.003346,0.000000,0.012710,0.321173,0.562876,1.030608e-01,0.000181,0.820707,0.866039,0.458142,1.200842,9072.5625,520507.9125,2.986240e+07,8.641616e+05,4.957838e+07,2.844394e+09,1.631876e+11,3.0,3.0
59,24.95,894.30,0.037366,4.447297e-01,0.471576,0.046328,2.170599e-08,0.000000,0.031877,0.425038,0.489533,0.053551,7.327987e-07,0.000000,0.040404,0.212313,-0.933763,-0.325363,384.1600,5868.2400,8.964036e+04,7.529536e+03,1.150175e+05,1.756951e+06,2.683832e+07,0.0,0.0
147,19.60,61.35,0.000000,1.246894e-03,0.161238,0.594552,2.381131e-01,0.004849,0.013129,0.324451,0.561102,0.101153,1.647832e-04,0.000000,0.845328,0.435052,0.505980,-0.087210,9574.6225,108163.3900,1.221909e+06,9.368768e+05,1.058379e+07,1.195638e+08,1.350698e+09,3.0,0.0
481,19.65,225.75,0.000000,8.378857e-03,0.281917,0.581177,1.280308e-01,0.000496,0.000027,0.075086,0.529717,0.374341,2.082981e-02,0.000000,0.691358,0.740657,0.271389,0.668277,7242.0100,311657.4750,1.341208e+07,6.162951e+05,2.652205e+07,1.141368e+09,4.911837e+10,3.0,2.0
1001,95.35,660.90,0.000000,3.222760e-07,0.051174,0.483935,4.313419e-01,0.033548,0.000000,0.000002,0.058025,0.499346,4.135952e-01,0.029031,0.993401,0.994177,0.841766,2.041736,13479.2100,964854.8550,6.906524e+07,1.564936e+06,1.120196e+08,8.018475e+09,5.739701e+11,4.0,4.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7040,103.20,7362.90,0.014964,3.379715e-01,0.553403,0.093553,1.084496e-04,0.000000,0.030423,0.419306,0.494504,0.055766,1.362306e-06,0.000000,0.232065,0.232004,-0.749770,-0.311461,876.1600,10254.9200,1.200276e+05,2.593434e+04,3.035456e+05,3.552817e+06,4.158356e+07,0.0,0.0
7041,29.60,346.45,0.000000,2.528244e-02,0.396958,0.512832,6.492004e-02,0.000008,0.031651,0.424163,0.490299,0.053885,8.109558e-07,0.000000,0.541667,0.215486,0.074517,-0.323235,5535.3600,22811.0400,9.400356e+04,4.118308e+05,1.697141e+06,6.993865e+06,2.882149e+07,2.0,0.0
7042,74.40,306.60,0.000000,1.924552e-04,0.104290,0.563987,3.190834e-01,0.012447,0.000000,0.001356,0.164895,0.595424,2.337617e-01,0.004564,0.935269,0.946435,0.649494,1.608554,11161.9225,723121.4250,4.684718e+07,1.179257e+06,7.639778e+07,4.949405e+09,3.206455e+11,4.0,3.0
7043,105.65,6844.50,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [15]:
numeric_transformer = ColumnTransformer(
    transformers=[
        ('spline', SplineTransformer(n_knots=n_knots, degree=degree_spline), num_columns),
        ('quantile', QuantileTransformer(n_quantiles=n_quantiles), num_columns),
        ('robust', RobustScaler(), num_columns),
        ('polynomial', PolynomialFeatures(degree=degree), num_columns),
        ('kbins', KBinsDiscretizer(n_bins=n_bins, encode=encode, strategy=strategy, subsample=subsample), num_columns),
    ]
)

categorical_transformer = Pipeline(
	steps=[
        ('onehot',OneHotEncoder(categories='auto', handle_unknown='ignore', max_categories=10, sparse_output=False, drop='first'))
    ]
)

preprocessor = ColumnTransformer(
    transformers=[
	('num',numeric_transformer,num_columns),
    ('cat',categorical_transformer,cat_columns)
    ]
)

encoded_features = preprocessor.fit_transform(df)

transformed_df = pd.DataFrame(encoded_features, columns=preprocessor.get_feature_names_out())

df = pd.concat([df.reset_index(drop=True), transformed_df.reset_index(drop=True)], axis=1)
df.head(2)

,type,paperless_billing,payment_method,monthly_charges,total_charges,internet_service,online_security,online_backup,device_protection,tech_support,streaming_tv,streaming_movies,gender,senior_citizen,partner,dependents,multiple_lines,target,num__spline__monthly_charges_sp_0,num__spline__monthly_charges_sp_1,num__spline__monthly_charges_sp_2,num__spline__monthly_charges_sp_3,num__spline__monthly_charges_sp_4,num__spline__monthly_charges_sp_5,num__spline__total_charges_sp_0,num__spline__total_charges_sp_1,num__spline__total_charges_sp_2,num__spline__total_charges_sp_3,num__spline__total_charges_sp_4,num__spline__total_charges_sp_5,num__quantile__monthly_charges,num__quantile__total_charges,num__robust__monthly_charges,num__robust__total_charges,num__polynomial__1,num__polynomial__monthly_charges,num__polynomial__total_charges,num__polynomial__monthly_charges^2,num__polynomial__monthly_charges total_charges,num__polynomial__total_charges^2,num__polynomial__monthly_charges^3,num__polynomial__monthly_charges^2 total_charges,num__polynomial__monthly_charges total_charges^2,num__polynomial__total_charges^3,num__kbins__monthly_charges,num__kbins__total_charges,cat__type_One year,cat__type_Two year,cat__payment_method_Credit card (automatic),cat__payment_method_Electronic check,cat__payment_method_Mailed check,cat__internet_service_Fiber optic,cat__gender_Male
0,One year,No,Mailed check,20.65,1022.95,Fiber optic,No,No,No,No,No,No,Female,0,No,No,No,0,0.034259,0.433936,0.481590,0.050214,2.168151e-07,0.0,0.014515,0.334777,0.555277,0.095311,0.000120,0.0,0.151515,0.415985,-0.914443,-0.111572,1.0,20.65,1022.95,426.4225,21123.9175,1.046427e+06,8805.624625,436208.896375,2.160871e+07,1.070442e+09,0.0,0.0,1.0,0.0,0.0,0.0,1.0,1.0,0.0
1,Two year,No,Credit card (automatic),24.95,894.30,Fiber optic,No,No,No,No,No,No,Female,0,Yes,Yes,Yes,0,0.023507,0.388355,0.519449,0.068676,1.316872e-05,0.0,0.016892,0.350947,0.545446,0.086646,0.000069,0.0,0.196970,0.387206,-0.835327,-0.149584,1.0,24.95,894.30,622.5025,22312.7850,7.997725e+05,15531.437375,556703.985750,1.995432e+07,7.152365e+08,0.0,0.0,0.0,1.0,1.0,0.0,0.0,1.0,0.0


In [ ]:
#mlflow.set_tracking_uri(f"http://{TRACKING_SERVER_HOST}:{TRACKING_SERVER_PORT}")
#mlflow.set_registry_uri(f"http://{TRACKING_SERVER_HOST}:{TRACKING_SERVER_PORT}")

#experiment_id = mlflow.get_experiment_by_name(EXPERIMENT_NAME).experiment_id

#with mlflow.start_run(run_name=RUN_NAME, experiment_id=experiment_id) as run:
#    run_id = run.info.run_id

#    mlflow.sklearn.log_model(preprocessor, "column_transformer") 
    
#    print(f"Run ID: {run_id}")

In [17]:
df.head(2)

,type,paperless_billing,payment_method,monthly_charges,total_charges,internet_service,online_security,online_backup,device_protection,tech_support,streaming_tv,streaming_movies,gender,senior_citizen,partner,dependents,multiple_lines,target,num__spline__monthly_charges_sp_0,num__spline__monthly_charges_sp_1,num__spline__monthly_charges_sp_2,num__spline__monthly_charges_sp_3,num__spline__monthly_charges_sp_4,num__spline__monthly_charges_sp_5,num__spline__total_charges_sp_0,num__spline__total_charges_sp_1,num__spline__total_charges_sp_2,num__spline__total_charges_sp_3,num__spline__total_charges_sp_4,num__spline__total_charges_sp_5,num__quantile__monthly_charges,num__quantile__total_charges,num__robust__monthly_charges,num__robust__total_charges,num__polynomial__1,num__polynomial__monthly_charges,num__polynomial__total_charges,num__polynomial__monthly_charges^2,num__polynomial__monthly_charges total_charges,num__polynomial__total_charges^2,num__polynomial__monthly_charges^3,num__polynomial__monthly_charges^2 total_charges,num__polynomial__monthly_charges total_charges^2,num__polynomial__total_charges^3,num__kbins__monthly_charges,num__kbins__total_charges,cat__type_One year,cat__type_Two year,cat__payment_method_Credit card (automatic),cat__payment_method_Electronic check,cat__payment_method_Mailed check,cat__internet_service_Fiber optic,cat__gender_Male
0,One year,No,Mailed check,20.65,1022.95,Fiber optic,No,No,No,No,No,No,Female,0,No,No,No,0,0.034259,0.433936,0.481590,0.050214,2.168151e-07,0.0,0.014515,0.334777,0.555277,0.095311,0.000120,0.0,0.151515,0.415985,-0.914443,-0.111572,1.0,20.65,1022.95,426.4225,21123.9175,1.046427e+06,8805.624625,436208.896375,2.160871e+07,1.070442e+09,0.0,0.0,1.0,0.0,0.0,0.0,1.0,1.0,0.0
1,Two year,No,Credit card (automatic),24.95,894.30,Fiber optic,No,No,No,No,No,No,Female,0,Yes,Yes,Yes,0,0.023507,0.388355,0.519449,0.068676,1.316872e-05,0.0,0.016892,0.350947,0.545446,0.086646,0.000069,0.0,0.196970,0.387206,-0.835327,-0.149584,1.0,24.95,894.30,622.5025,22312.7850,7.997725e+05,15531.437375,556703.985750,1.995432e+07,7.152365e+08,0.0,0.0,0.0,1.0,1.0,0.0,0.0,1.0,0.0


In [18]:
# Обучение модели CatBoostClassifier
X = transformed_df
y = df['target']  # Целевая переменная

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [19]:
print(f"Размер выборки для обучения: {X_train.shape}")
print(f"Размер выборки для теста: {X_test.shape}")

Размер выборки для обучения: (5634, 35)
Размер выборки для теста: (1409, 35)


In [20]:
loss_function = "Logloss"
task_type = 'CPU'
random_seed = 0
iterations = 300
verbose = False

In [21]:
params = {
    'learning_rate': [0.01, 0.05, 0.1],
    'depth': [4, 6, 8],
    'l2_leaf_reg': [1, 3, 5]
}

In [22]:
model = CatBoostClassifier(
    loss_function=loss_function,
    task_type=task_type,
    random_seed=random_seed,
    iterations=iterations,
    verbose=verbose
)

In [26]:
cv = GridSearchCV(
    estimator=model,
    param_grid=params,
    scoring='roc_auc',
    cv=2,
    n_jobs=-1,
    verbose=0
)

In [27]:
clf = cv.fit(X_train, y_train)

In [29]:
#os.environ["MLFLOW_S3_ENDPOINT_URL"] = "https://storage.yandexcloud.net"
#os.environ["AWS_ACCESS_KEY_ID"] = os.getenv("S3_ACCESS_KEY")
#os.environ["AWS_SECRET_ACCESS_KEY"] = os.getenv("S3_SECRET_KEY")


mlflow.set_tracking_uri(f"http://{TRACKING_SERVER_HOST}:{TRACKING_SERVER_PORT}")
mlflow.set_registry_uri(f"http://{TRACKING_SERVER_HOST}:{TRACKING_SERVER_PORT}")

In [30]:
cv_results = pd.DataFrame(clf.cv_results_)

best_params = clf.best_params_

model_best = CatBoostClassifier(
    **best_params,
    loss_function=loss_function,
    task_type=task_type,
    random_seed=random_seed,
    iterations=iterations,
    verbose=verbose
)

In [31]:
model_best.fit(X_train, y_train)

prediction = model_best.predict(X_test)
probas = model_best.predict_proba(X_test)[:, 1]

In [32]:
# расчёт метрик качества
metrics = {}

# Метрики качества
_, err1, _, err2 = confusion_matrix(y_test, prediction, normalize='all').ravel()
auc = roc_auc_score(y_test, probas)
precision = precision_score(y_test, prediction)
recall = recall_score(y_test, prediction)
f1 = f1_score(y_test, prediction)
logloss = log_loss(y_test, prediction)

# сохранение метрик в словарь
metrics["err1"] = err1
metrics["err2"] = err2
metrics["auc"] = auc
metrics["precision"] = precision
metrics["recall"] = recall
metrics["f1"] = f1
metrics["logloss"] = logloss

# дополнительные метрики из результатов кросс-валидации
metrics['mean_fit_time'] = cv_results['mean_fit_time'].mean()
metrics['std_fit_time'] = cv_results['std_fit_time'].mean()
metrics['mean_test_score'] = cv_results['mean_test_score'].mean()
metrics['std_test_score'] = cv_results['std_test_score'].mean()
metrics['best_score'] = clf.best_score_

In [33]:
# настройки для логирования в MLFlow
pip_requirements = '../requirements.txt'
signature = mlflow.models.infer_signature(X_test, prediction)
input_example = X_test[:10]

experiment_id = mlflow.get_experiment_by_name(EXPERIMENT_NAME).experiment_id

In [36]:
# Логирование в MLflow
with mlflow.start_run(run_name=RUN_NAME, experiment_id=experiment_id) as run:
    run_id = run.info.run_id
    mlflow.sklearn.log_model(cv, artifact_path='cv')
    mlflow.log_params(best_params)
    mlflow.log_metrics(metrics)
    mlflow.catboost.log_model(
        cb_model=model_best,
        artifact_path='models',
        registered_model_name=REGISTRY_MODEL_NAME,
        signature=signature,
        input_example=input_example,
        await_registration_for=60,
        pip_requirements=pip_requirements
    )


print("Run ID:", run_id)
print("Model logged to MLflow with ID:", model_info.model_uri)
print(f"✅ Model registered successfully!")

2025/05/01 19:15:04 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.
2025/05/01 19:15:05 WARNING mlflow.utils.requirements_utils: Detected one or more mismatches between the model's dependencies and the current Python environment:
 - mlflow (current: 2.20.3, required: mlflow==2.7.1)
 - boto3 (current: 1.33.1, required: boto3==1.34.77)
To fix the mismatches, call `mlflow.pyfunc.get_model_dependencies(model_uri)` to fetch the model's environment and install dependencies using the resulting environment file.
2025/05/01 19:15:05 WARNING mlflow.utils.requirements_utils: Detected one or more mismatches between the model's dependencies and the current Python environment:
 - mlflow (current: 2.20.3, required: mlflow==2.7.1)
 - boto3 (current: 1.33.1, required: boto3==1.34.77)
To fix the mismatches, call `mlflow.pyfunc.get_model_dependencies(model_uri)` to fetch the mod

🏃 View run model_grid_search at: http://127.0.0.1:5000/#/experiments/4/runs/0f04b985a1c74131924ca5365f9db30a
🧪 View experiment at: http://127.0.0.1:5000/#/experiments/4
Run ID: 0f04b985a1c74131924ca5365f9db30a


Created version '7' of model 'churn_model_martynov_alexey'.


NameError: name 'model_info' is not defined

2025/05/01 19:15:04 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.
2025/05/01 19:15:05 WARNING mlflow.utils.requirements_utils: Detected one or more mismatches between the model's dependencies and the current Python environment:
 - mlflow (current: 2.20.3, required: mlflow==2.7.1)
 - boto3 (current: 1.33.1, required: boto3==1.34.77)
To fix the mismatches, call `mlflow.pyfunc.get_model_dependencies(model_uri)` to fetch the model's environment and install dependencies using the resulting environment file.
2025/05/01 19:15:05 WARNING mlflow.utils.requirements_utils: Detected one or more mismatches between the model's dependencies and the current Python environment:
 - mlflow (current: 2.20.3, required: mlflow==2.7.1)
 - boto3 (current: 1.33.1, required: boto3==1.34.77)
To fix the mismatches, call `mlflow.pyfunc.get_model_dependencies(model_uri)` to fetch the model's environment and install dependencies using the resulting environment file.
Registered model 'churn_model_martynov_alexey' already exists. Creating a new version of this model...
2025/05/01 19:15:06 INFO mlflow.store.model_registry.abstract_store: Waiting up to 60 seconds for model version to finish creation. Model name: churn_model_martynov_alexey, version 7
🏃 View run model_grid_search at: http://127.0.0.1:5000/#/experiments/4/runs/0f04b985a1c74131924ca5365f9db30a
🧪 View experiment at: http://127.0.0.1:5000/#/experiments/4
Run ID: 0f04b985a1c74131924ca5365f9db30a
Created version '7' of model 'churn_model_martynov_alexey'.